# 2.1. Advanced Python Constructs

## 2.1.1 Iterators, generator expressions and generators

### 2.1.1 Iterators

An iterator is an object adhering to the iterator protocol — basically this means that it has a next method, which, when called, returns the next item in the sequence, and when there’s nothing to return, raises the StopIteration exception.

An iterator object allows to loop just once. It holds the state (position) of a single iteration, or from the other side, each loop over a sequence requires a single iterator object. This means that we can iterate over the same sequence more than once concurrently. Separating the iteration logic from the sequence allows us to have more than one way of iteration.

Calling the __iter__ method on a container to create an iterator object is the most straightforward way to get hold of an iterator. The iter function does that for us, saving a few keystrokes.

In [1]:
nums = [1, 2, 3]

In [2]:
iter(nums)

In [3]:
nums.__iter__()

In [4]:
nums.__reversed__()

In [7]:
it = iter(nums)

In [8]:
next(it)

1

In [9]:
next(it)

2

In [10]:
next(it)

3

In [11]:
next(it)

StopIteration: 

When used in a loop, `StopIteration` is swallowed and causes the loop to finish. But with explicit invocation, we can see that once the iterator is exhausted, accessing it raises an exception.

Using the for..in loop also uses the `__iter__` method. This allows us to transparently start the iteration over a sequence. But if we already have the iterator, we want to be able to use it in an for loop in the same way. In order to achieve this, iterators in addition to next are also required to have a method called `__iter__` which returns the iterator (`self`).

Support for iteration is pervasive in Python: all sequences and unordered containers in the standard library allow this. The concept is also stretched to other things: e.g. `file` objects support iteration over lines.

### 2.1.1.2 Generator expressions

A second way in which iterator objects are created is through generator expressions, the basis for list comprehensions. To increase clarity, a generator expression must always be enclosed in parentheses or an expression. If round parentheses are used, then a generator iterator is created. If rectangular parentheses are used, the process is short-circuited and we get a `list`.

In [12]:
(i for i in nums)

<generator object <genexpr> at 0x000001A8A3517048>

In [13]:
[i for i in nums]

[1, 2, 3]

In [14]:
list(i for i in nums)

[1, 2, 3]

The list comprehension syntax also extends to dictionary and set comprehensions. A `set` is created when the generator expression is enclosed in curly braces. A `dict` is created when the generator expression contains “pairs” of the form `key:value`:

In [15]:
{i for i in range(0, 3)}

{0, 1, 2}

In [16]:
{i:i**2 for i in range(0, 3)}

{0: 0, 1: 1, 2: 4}

One gotcha should be mentioned: in old Pythons the index variable (i) would leak, and in versions >= 3 this is fixed.